In [ ]:
#@title Installing foundry, cloning repo, installing python dependencies
!curl -L https://foundry.paradigm.xyz | bash
!source ~/.bashrc
import os
os.environ['PATH'] += ":/root/.foundry/bin"
!foundryup

!git clone -b feat/scripts-and-docs https://github.com/baking-bad/etherlink-bridge.git
%cd etherlink-bridge/
!pip install poetry
!poetry install

### Environment Initialization:

Run the following cell to set up the environment and provide private keys for interacting *with* the bridge. Default values are provided for all variables; however, you may wish to deploy your own token and establish a bridge using your own keys.

In [ ]:
!poetry run init_wallets

Enter "L1_PRIVATE_KEY" (default: edsk2nu78mRwg4V5Ka7XCJFVbVPPwhry8YPeEHRwzGQHEpGAffDvrH): 
Enter "L1_PUBLIC_KEY_HASH" (default: tz1YG6P2GTQKFpd9jeuESam2vg6aA9HHRkKo): 
Enter "L1_RPC_URL" (default: https://rpc.tzkt.io/nairobinet/): 
Enter "L1_ROLLUP_ADDRESS" (default: sr1Cz5HkqyC8ozTYbDij8YmseAAG3SgZtpRb): 
Enter "L2_PRIVATE_KEY" (default: 8636c473b431be57109d4153735315a5cdf36b3841eb2cfa80b75b3dcd2d941a): 
Enter "L2_PUBLIC_KEY" (default: 0xBefD2C6fFC36249ebEbd21d6DF6376ecF3BAc448): 
Enter "L2_MASTER_KEY" (default: 9722f6cc9ff938e63f8ccb74c3daa6b45837e5c5e3835ac08c44c50ab5f39dc0): 
Enter "L2_RPC_URL" (default: https://etherlink.dipdup.net): 
Enter "L2_KERNEL_ADDRESS" (default: 0x0000000000000000000000000000000000000000): 
Enter "L2_WITHDRAW_PRECOMPILE_ADDRESS" (default: 0x0000000000000000000000000000000000000040): 


### Bridge token pair listing:

The **Ticket Transport Layer**, a key component of the bridge, enables ticket transfers between Tezos and Etherlink. To list a new token pair and establish a bridge, users need to:
1. If the token is an **FA2** or **FA1.2** standard token that doesn’t natively support tickets, the user must deploy a **Ticketer** contract on the Tezos side. This contract links to the specific Tezos token to convert it into a ticket.
2. The user then deploys an **ERC20Proxy** on the Etherlink side – a smart contract implementing ERC20. This contract should be configured to expect tickets from the deployed **Ticketer** including ticketer address and ticket content. This setup allows the rollup to mint ERC20 tokens corresponding to the incoming tickets from Tezos.
3. Additionally, deploy a **TicketHelper** on the Tezos side, targeting the specific token and **Ticketer** pair. This step is necessary as the **transfer_ticket** operation is not currently supported by wallets. The **TicketHelper** facilitates the wrapping of tokens into tickets and their transfer to the rollup in a single transaction.

#### Deploying a Token

For demonstration purposes, users can deploy a test token that will later be bridged. The bridge has been tested with two types of tokens, which are available in the repository:
- The **FA1.2** **Ctez** token.
- The **FA2** **fxhash** token.

To deploy this token and allocate the total supply to the token's originator, execute the following script. The example below illustrates the deployment for an **FA1.2** token type:

In [ ]:
!poetry run deploy_token --token-type FA1.2

deploying contract from filename /content/etherlink-bridge/tezos/tests/helpers/../tokens/fa12-Ctez.tz
found contract address: KT19SS4BWZoaZ6xG3ozXNtNdrebKTMn6YM5y


#### Deploying a Ticketer

To deploy a ticketer for a specific token address and token id, use the following script. The example below uses a **FA1.2** token previously deployed on Nairobinet:

In [ ]:
!poetry run deploy_ticketer --token-address KT19SS4BWZoaZ6xG3ozXNtNdrebKTMn6YM5y --token-type FA1.2 --token-id 0

deploying contract from filename /content/etherlink-bridge/tezos/tests/helpers/../../build/ticketer.tz
found contract address: KT1KtbR9fr5Lu2r6VVKosG5XbXewVhegGc7L
address_bytes: 017c08663c9a107efae948c69296376b3ba08d279900
content_bytes: 0707000005090a0000005f05020000005907040100000010636f6e74726163745f616464726573730a0000001c050a00000016010964830cb917ea6316a180f0ec12e08ae2529457000704010000000a746f6b656e5f747970650a0000000b0501000000054641312e32


During the Ticketer's deployment, user will receive its parameters, including **address_bytes** and **content_bytes**. These are required for the origination of the **ERC20Proxy**.

#### Deploying a Ticket Helper

To enable Tezos wallets to interact with specific tickets, deploy a Ticket Helper using the script below. The example provided uses a ticketer that was previously deployed in Nairobinet:

In [ ]:
!poetry run deploy_ticket_helper --ticketer-address KT1KtbR9fr5Lu2r6VVKosG5XbXewVhegGc7L

deploying contract from filename /content/etherlink-bridge/tezos/tests/helpers/../../build/ticket-helper.tz
found contract address: KT1D8MV1ZoxAK3usKRzgTcSgwPm3z2Ywi8UR


#### Deploying ERC20Proxy

Finally, to deploy a token contract on the Etherlink side, which will mint tokens upon deposit, execute the script below. This script requires **ticketer-address-bytes** and **ticketer-content-bytes**. The example uses bytes from a ticketer previously deployed in Nairobinet:

In [ ]:
!poetry run deploy_erc20 --ticketer-address-bytes 017c08663c9a107efae948c69296376b3ba08d279900 --ticketer-content-bytes 0707000005090a0000005f05020000005907040100000010636f6e74726163745f616464726573730a0000001c050a00000016010964830cb917ea6316a180f0ec12e08ae2529457000704010000000a746f6b656e5f747970650a0000000b0501000000054641312e32 --token-name "FA1.2/2024" --token-symbol "2024" --decimals 0

NOTE: To obtain the **ticketer-address-bytes** and **content-bytes** for an already deployed ticketer, user can use the script below:

In [ ]:
!poetry run get_ticketer_params --ticketer-address KT1MkNDvrW4V6TY2MU8RhHWDVHm5y3B1sn5m

### Deposit

To make a deposit, user need to transfer Tickets to the rollup address with attached Routing Info in the [specified format](https://gitlab.com/baking-bad/tzip/-/blob/wip/029-etherlink-token-bridge/drafts/current/draft-etherlink-token-bridge/etherlink-token-bridge.md#deposit): `| receiver | proxy |` 40 bytes payload, both receiver and proxy are standard Ethereum addresses in raw form (H160). A script is available to facilitate this process. It requires the Ticket Helper address as **ticket-helper-address**, the Etherlink ERC20Proxy contract as **proxy-address**, and the bridged amount as the **amount** variable. Here's an example:

In [ ]:
!poetry run deposit --ticket-helper-address KT1CXMcdSCw3sviupN6hwzzmfcyMYAF58W77 --proxy-address 0x6065534feE55f585C2d43f8e78BcE4C308EE066B --amount 42

NOTE: This script performs two operations in bulk. The first operation approves the token for the **Ticket Helper**, and the second operation makes a call to the **Ticket Helper** **deposit** entrypoint.

### Withdrawal Process

The withdrawal process involves two key steps:
1. Initiating the withdrawal on the Etherlink side, which results in the creation of an outbox message in the rollup commitment.
2. Completing the withdrawal by executing the outbox message on the Tezos side once the commitment has settled (TODO: Clarify terminology if necessary).

#### Etherlink Withdrawal

To initiate a withdrawal, user need to call the withdrawal precompile on the Etherlink side. This requires providing the **ERC20Proxy** address along with Routing Info in a [specific format](https://gitlab.com/baking-bad/tzip/-/blob/wip/029-etherlink-token-bridge/drafts/current/draft-etherlink-token-bridge/etherlink-token-bridge.md#withdrawal), two forged contracts concatenated: `| receiver | proxy |` 44 bytes. Forged contact consists of binary suffix/prefix and body (blake2b hash digest). A script is available to facilitate the withdrawal process. It requires the ERC20 contract (which will burn tokens) as **proxy-address**, the Tezos router address (which will receive the ticket from the rollup) as **router-address**, and the bridged amount as **amount**. Additionally, the **ticketer-address-bytes** and **ticketer-content-bytes** are required to allow **ERC20Proxy** validate token before burning.

NOTE: For automatic unwrapping of Tezos tickets back into tokens, the **Ticketer** address can be provided as the **router-address**.

In [ ]:
!poetry run withdraw --proxy 0x6065534feE55f585C2d43f8e78BcE4C308EE066B --amount --router KT1CXMcdSCw3sviupN6hwzzmfcyMYAF58W77 42 --ticketer-address-bytes 01906a4f54a7c9bc66c7c159f5ab7df90166acb02b00 --ticketer-content-bytes 0707000005090a0000005f05020000005907040100000010636f6e74726163745f616464726573730a0000001c050a0000001601f923adf742bdc1c3e14944859de0afa800f6b7a5000704010000000a746f6b656e5f747970650a0000000b0501000000054641312e32

#### Finalizing Tezos Withdrawal

To complete the withdrawal process, user need to call the outbox message on the Tezos side once it has been finalized (settled) on the L1 side. During the withdrawal process, the Etherlink side emits a **Withdrawal** event, which includes **outboxLevel** and **outboxMsgId**.

TODO: Provide a link to the Blockscout page featuring this event.

TODO: Explain how to parse this information.

In [ ]:
!poetry get_proof --outbox-level 123456 --outbox-message-id 0

For executing the outbox message, run:

In [ ]:
!poetry execute_outbox_message --commitment {COMMITMENT} --proof {PROOF}

TODO: fill shell script with real parameters